In [1]:
import os
import glob
import shutil
from lxml import etree
from PIL import Image

# ==========================================
# 1. ตั้งค่า Path และ Parameter
# ==========================================
# โฟลเดอร์ข้อมูลต้นฉบับ
source_root_dir = os.path.join("data", "500 cases with annotation-raw")

# โฟลเดอร์ปลายทางที่จะสร้าง (สำหรับเทรน YOLO)
output_dir = "yolo_dataset"
images_out_dir = os.path.join(output_dir, "images")
labels_out_dir = os.path.join(output_dir, "labels")

# สร้างโฟลเดอร์ถ้ายังไม่มี
os.makedirs(images_out_dir, exist_ok=True)
os.makedirs(labels_out_dir, exist_ok=True)

# Namespace ของ AIM XML
ns = {
    "aim": "gme://caCORE.caCORE/4.4/edu.northwestern.radiology.AIM"
}

# ==========================================
# 2. ฟังก์ชันช่วยอ่าน Polygon
# ==========================================
def get_polygon_points(xml_path):
    """
    อ่านไฟล์ XML แล้วคืนค่าเป็น List ของ (x, y) ที่เรียงตามลำดับแล้ว
    """
    try:
        tree = etree.parse(xml_path)
        root = tree.getroot()
        coords = []
        
        for point in root.findall(".//aim:TwoDimensionSpatialCoordinate", namespaces=ns):
            try:
                # อ่าน Index, X, Y
                idx_elem = point.findall("aim:coordinateIndex", namespaces=ns)
                x_elem = point.findall("aim:x", namespaces=ns)
                y_elem = point.findall("aim:y", namespaces=ns)
                
                if idx_elem and x_elem and y_elem:
                    index = int(idx_elem[0].get('value'))
                    x = float(x_elem[0].get('value'))
                    y = float(y_elem[0].get('value'))
                    coords.append((index, x, y))
            except:
                continue
        
        # เรียงตาม Index เพื่อให้เส้นลากต่อกันถูกต้อง
        coords.sort(key=lambda c: c[0])
        
        # ตัด Index ทิ้ง เอาแค่ (x, y)
        return [(c[1], c[2]) for c in coords]

    except Exception as e:
        print(f"Error reading XML {xml_path}: {e}")
        return []

# ==========================================
# 3. เริ่มกระบวนการแปลง (Main Loop)
# ==========================================
print(f"Starting conversion from: {source_root_dir}")

case_folders = sorted([d for d in os.listdir(source_root_dir) if os.path.isdir(os.path.join(source_root_dir, d))])
print(f"Found {len(case_folders)} cases.")

for case_name in case_folders:
    case_full_path = os.path.join(source_root_dir, case_name)
    
    # A. หาไฟล์รูปภาพ (.png)
    image_files = glob.glob(os.path.join(case_full_path, "*.png"))
    if not image_files:
        continue
    
    src_img_path = image_files[0]
    
    # เปิดภาพเพื่อเอาขนาด (W, H)
    try:
        with Image.open(src_img_path) as img:
            img_w, img_h = img.size
    except Exception as e:
        print(f"Error opening image {src_img_path}: {e}")
        continue

    # B. เตรียมชื่อไฟล์ปลายทาง
    # ตั้งชื่อไฟล์ใหม่ให้ unique เช่น case1.png, case1.txt
    dst_img_filename = f"{case_name}.png"
    dst_txt_filename = f"{case_name}.txt"
    
    dst_img_path = os.path.join(images_out_dir, dst_img_filename)
    dst_txt_path = os.path.join(labels_out_dir, dst_txt_filename)

    # C. หาไฟล์ XML ทั้งหมดในเคสนั้น
    xml_files = glob.glob(os.path.join(case_full_path, "*.xml"))
    
    yolo_lines = []
    
    for xml_file in xml_files:
        points = get_polygon_points(xml_file)
        
        if len(points) > 2: # Polygon ต้องมีอย่างน้อย 3 จุด
            # สร้าง String สำหรับ YOLO Segmentation Format
            # Format: <class-id> <x1_norm> <y1_norm> <x2_norm> <y2_norm> ...
            
            # ใช้ Class ID = 0 (Dental Caries)
            line_parts = ["0"] 
            
            for (px, py) in points:
                # Normalize Coordinate (หารด้วยขนาดภาพ)
                norm_x = px / img_w
                norm_y = py / img_h
                
                # Clip ค่าให้อยู่ระหว่าง 0-1 (กันพลาด)
                norm_x = max(0.0, min(1.0, norm_x))
                norm_y = max(0.0, min(1.0, norm_y))
                
                line_parts.append(f"{norm_x:.6f}")
                line_parts.append(f"{norm_y:.6f}")
            
            yolo_lines.append(" ".join(line_parts))

    # D. บันทึกไฟล์
    # 1. ถ้ามีข้อมูล Annotation ให้เขียนไฟล์ .txt และ copy รูปภาพ
    if yolo_lines:
        # Save .txt
        with open(dst_txt_path, "w") as f:
            f.write("\n".join(yolo_lines))
        
        # Copy .png ไปที่ folder yolo_dataset/images
        shutil.copy2(src_img_path, dst_img_path)
        
        # print(f"Converted {case_name}: {len(yolo_lines)} lesions.")

print("-" * 50)
print(f"Conversion Complete!")
print(f"Images saved to: {images_out_dir}")
print(f"Labels saved to: {labels_out_dir}")

Starting conversion from: data/500 cases with annotation-raw
Found 500 cases.
--------------------------------------------------
Conversion Complete!
Images saved to: yolo_dataset/images
Labels saved to: yolo_dataset/labels


---
---

# Train Test Split

```text
datasets/
└── dental_caries/
    ├── train/
    │   ├── images/
    │   └── labels/
    ├── valid/   (หรือ val)
    │   ├── images/
    │   └── labels/
    └── test/
        ├── images/
        └── labels/
```

In [2]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# --- ตั้งค่า ---
source_img_dir = "yolo_dataset/images"
source_lbl_dir = "yolo_dataset/labels"
root_dir = "datasets/dental_caries" # โฟลเดอร์ปลายทางที่จะใช้เทรนจริง

# สัดส่วน (Train 70%, Val 20%, Test 10%)
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# สร้างโฟลเดอร์ปลายทาง
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(root_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(root_dir, split, 'labels'), exist_ok=True)

# 1. รวบรวมรายชื่อไฟล์ (อิงจากไฟล์รูปภาพเป็นหลัก)
all_files = [f for f in os.listdir(source_img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
all_files.sort() # เรียงเพื่อให้ผลเหมือนเดิมทุกครั้งถ้า seed เท่าเดิม

# 2. แบ่งข้อมูล (Splitting)
# แบ่ง Train ออกมาก่อน
train_files, rest_files = train_test_split(all_files, train_size=train_ratio, random_state=42)
# แบ่ง Val และ Test จากส่วนที่เหลือ
val_files, test_files = train_test_split(rest_files, test_size=test_ratio/(val_ratio+test_ratio), random_state=42)

print(f"Total files: {len(all_files)}")
print(f"Train: {len(train_files)}, Valid: {len(val_files)}, Test: {len(test_files)}")

# 3. ฟังก์ชันย้ายไฟล์
def copy_files(file_list, split_name):
    for filename in file_list:
        # ชื่อไฟล์รูปและ label (ตัดนามสกุลออกแล้วเติม .txt)
        base_name = os.path.splitext(filename)[0]
        lbl_name = base_name + ".txt"
        
        src_img = os.path.join(source_img_dir, filename)
        src_lbl = os.path.join(source_lbl_dir, lbl_name)
        
        dst_img = os.path.join(root_dir, split_name, 'images', filename)
        dst_lbl = os.path.join(root_dir, split_name, 'labels', lbl_name)
        
        # Copy รูป
        shutil.copy2(src_img, dst_img)
        
        # Copy Label (ถ้ามี เพราะบางรูปอาจไม่มี detection เลย ก็ต้อง copy รูปไปเป็น background image)
        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)

# เริ่มย้าย
print("Copying files...")
copy_files(train_files, 'train')
copy_files(val_files, 'valid')
copy_files(test_files, 'test')
print("Data splitting complete!")

Total files: 500
Train: 350, Valid: 100, Test: 50
Copying files...
Data splitting complete!


---
---

- เราต้องไป label ใหม่ เพราะตอนนี้ในไฟล์ label มา มีแต่เลข 0 แปลว่าฟันผุ แต่ไม่ได้บอกว่าฟันซี่ไหน ตาม FDI แลพด้านไหน
- จำนวน calss nc: 3*32 (ด้าน x จำนวนซี่)


- ถ้าเราซอยละเอียดเกินเป็น 96 case
- ถ้าทำ ทีเดียวมันอาจจะ overfit มั้ย?
- ทำ 2 ชั้นดีมั้ย?
    - ทำผุ ซ้าย ขวา บน ก่อน -> แล้วค่อยหาว่าเป็นของซี่ไหน (ในใจ)
    - หรือ หาซี่ก่อน แล้วค่อยทำผุ
    - หรือมีวิธีอื่นอีก ที่ควรทำ